In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report
import numpy as np

# ==== CONFIG ====
DATA_DIR = r"D:\Marine Data"  # Your dataset root folder
BATCH_SIZE = 32
IMG_SIZE = 224
NUM_EPOCHS = 15
LEARNING_RATE = 1e-3
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ==== 1. DATA PREPARATION ====
train_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# ImageFolder expects structure: root/class_x/*.jpg
dataset = datasets.ImageFolder(DATA_DIR, transform=train_transforms)

# Split train/val
val_split = 0.2
num_val = int(len(dataset) * val_split)
num_train = len(dataset) - num_val
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [num_train, num_val])

# Apply validation transform to val set
val_dataset.dataset.transform = val_transforms

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

NUM_CLASSES = len(dataset.classes)
print(f"✅ Dataset ready: {NUM_CLASSES} classes, {num_train} train, {num_val} val")

# ==== 2. MODEL ====
model = models.mobilenet_v3_large(weights=models.MobileNet_V3_Large_Weights.DEFAULT)
# Replace final classification layer
model.classifier[3] = nn.Linear(model.classifier[3].in_features, NUM_CLASSES)
model = model.to(DEVICE)

# ==== 3. LOSS & OPTIMIZER ====
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# ==== 4. TRAIN LOOP ====
best_val_acc = 0.0

for epoch in range(NUM_EPOCHS):
    model.train()
    train_loss, train_correct = 0.0, 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        train_correct += torch.sum(preds == labels).item()

    train_acc = train_correct / num_train

    # Validation
    model.eval()
    val_loss, val_correct = 0.0, 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            val_correct += torch.sum(preds == labels).item()

    val_acc = val_correct / num_val

    print(f"Epoch {epoch+1}/{NUM_EPOCHS} | "
          f"Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")

    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "mobilenetv3_best.pth")

print("✅ Training complete. Best Val Acc:", best_val_acc)


✅ Dataset ready: 87 classes, 33590 train, 8397 val


Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-5c1a4163.pth" to C:\Users\vkr30/.cache\torch\hub\checkpoints\mobilenet_v3_large-5c1a4163.pth
100%|██████████| 21.1M/21.1M [00:02<00:00, 9.63MB/s]


Epoch 1/15 | Train Acc: 0.6966 | Val Acc: 0.7755
Epoch 2/15 | Train Acc: 0.8197 | Val Acc: 0.8166
Epoch 3/15 | Train Acc: 0.8484 | Val Acc: 0.8145
Epoch 4/15 | Train Acc: 0.8679 | Val Acc: 0.8159
Epoch 5/15 | Train Acc: 0.8834 | Val Acc: 0.8335
Epoch 6/15 | Train Acc: 0.8987 | Val Acc: 0.8108
Epoch 7/15 | Train Acc: 0.9101 | Val Acc: 0.8246
Epoch 8/15 | Train Acc: 0.9188 | Val Acc: 0.8164
Epoch 9/15 | Train Acc: 0.9241 | Val Acc: 0.8445
Epoch 10/15 | Train Acc: 0.9347 | Val Acc: 0.8417
Epoch 11/15 | Train Acc: 0.9394 | Val Acc: 0.8373
Epoch 12/15 | Train Acc: 0.9432 | Val Acc: 0.8418
Epoch 13/15 | Train Acc: 0.9461 | Val Acc: 0.8308
Epoch 14/15 | Train Acc: 0.9533 | Val Acc: 0.8442
Epoch 15/15 | Train Acc: 0.9563 | Val Acc: 0.8555
✅ Training complete. Best Val Acc: 0.8555436465404311
